In [1]:
import numpy as np
import pandas as pd
import pyspark.pandas as ps

/usr/local/lib/python3.12/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


AttributeError: `np.NaN` was removed in the NumPy 2.0 release. Use `np.nan` instead.

In [15]:
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("v1_pd_series").master("spark://spark-master:7077") \
#        .getOrCreate()

### Object creation - Series

In [18]:
# Create pandas series...
pser = pd.Series([1,3,5,np.nan, 6,8])
print("Dtype : ",type(pser))
print(pser)

Dtype :  <class 'pandas.core.series.Series'>
0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64


In [19]:
# Create Spark pandas series
psser = ps.Series([3,1,5,np.nan,8,6])
print("Dtype : ",type(psser))
display(psser)

Dtype :  <class 'pyspark.pandas.series.Series'>


0    3.0
1    1.0
2    5.0
3    NaN
4    8.0
5    6.0
dtype: float64

In [20]:
psser_1 = ps.from_pandas(pser)
psser_1



0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [21]:
print(psser_1.sort_index())

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64


###  Object Creation - Dataframe

In [22]:
my_dict = {"A" : np.random.rand(5),
           "B" : np.random.rand(5)}
print(my_dict)

{'A': array([0.62238203, 0.64304826, 0.61254246, 0.16344178, 0.34931016]), 'B': array([0.08348756, 0.61524944, 0.63890046, 0.99464395, 0.91606447])}


In [23]:
pdf = pd.DataFrame(my_dict)
print(type(pdf))
pdf.head()

<class 'pandas.core.frame.DataFrame'>


,A,B
0,0.622382,0.083488
1,0.643048,0.615249
2,0.612542,0.638900
3,0.163442,0.994644
4,0.349310,0.916064


In [24]:
spdf = ps.DataFrame(pdf)
print(type(spdf))
display(spdf)

<class 'pyspark.pandas.frame.DataFrame'>


,A,B
0,0.622382,0.083488
1,0.643048,0.615249
2,0.612542,0.638900
3,0.163442,0.994644
4,0.349310,0.916064


In [25]:
spdf.sort_index()

,A,B
0,0.622382,0.083488
1,0.643048,0.615249
2,0.612542,0.638900
3,0.163442,0.994644
4,0.349310,0.916064


### View Data

In [26]:
spdf.head(2)

,A,B
0,0.622382,0.083488
1,0.643048,0.615249


In [28]:
# Summary statistics
spdf.describe()

,A,B
count,5.000000,5.000000
mean,0.478145,0.649669
std,0.213129,0.357714
min,0.163442,0.083488
25%,0.349310,0.615249
50%,0.612542,0.638900
75%,0.622382,0.916064
max,0.643048,0.994644


In [30]:
# Sort values method
spdf.sort_values(by='B')

,A,B
0,0.622382,0.083488
1,0.643048,0.615249
2,0.612542,0.638900
4,0.349310,0.916064
3,0.163442,0.994644


In [31]:
# Transpose method
spdf.transpose()

,0,1,2,3,4
A,0.622382,0.643048,0.612542,0.163442,0.349310
B,0.083488,0.615249,0.638900,0.994644,0.916064


In [32]:
ps.get_option("compute.max_rows")

1000

In [33]:
#change max_rows
ps.set_option("compute.max_rows", 2000)
ps.get_option("compute.max_rows")

2000

###  Data Selection

In [34]:
spdf["A"]

0    0.622382
1    0.643048
2    0.612542
3    0.163442
4    0.349310
Name: A, dtype: float64

In [37]:
spdf[["A","B"]]

,A,B
0,0.622382,0.083488
1,0.643048,0.615249
2,0.612542,0.638900
3,0.163442,0.994644
4,0.349310,0.916064


In [39]:
# index 0  to 3
spdf.loc[0:3]

,A,B
0,0.622382,0.083488
1,0.643048,0.615249
2,0.612542,0.638900
3,0.163442,0.994644


In [42]:
# Slicing
spdf.iloc[0:5, 0:1]

,A
0,0.622382
1,0.643048
2,0.612542
3,0.163442
4,0.349310


### Data Merge

In [43]:
psser = ps.Series([100,200,300,400,500],index=[0,1,2,3,4])
psser

0    100
1    200
2    300
3    400
4    500
dtype: int64

In [47]:
from pyspark.pandas.config import set_option,reset_option
set_option("compute.ops_on_diff_frames", True)

spdf["C"] = psser

#Reset to default to avoid potential expensive operation in the future
reset_option("compute.ops_on_diff_frames")
print(spdf)

          A         B    C
0  0.622382  0.083488  100
1  0.643048  0.615249  200
2  0.612542  0.638900  300
3  0.163442  0.994644  400
4  0.349310  0.916064  500


### Applying Python function with Pandas-on-Spark

In [48]:
spdf.apply(np.cumsum)

/usr/local/lib/python3.12/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,A,B,C
0,0.622382,0.083488,100
1,1.265430,0.698737,300
2,1.877973,1.337637,600
3,2.041415,2.332281,1000
4,2.390725,3.248346,1500


In [51]:
# cumulative sum along the columns.
spdf.apply(np.cumsum,axis=1)

/usr/local/lib/python3.12/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,A,B,C
0,0.622382,0.705870,100.705870
1,0.643048,1.258298,201.258298
2,0.612542,1.251443,301.251443
3,0.163442,1.158086,401.158086
4,0.349310,1.265375,501.265375


In [52]:
spdf.apply(lambda x : x ** 2)

/usr/local/lib/python3.12/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,A,B,C
0,0.387359,0.006970,10000
1,0.413511,0.378532,40000
2,0.375208,0.408194,90000
3,0.026713,0.989317,160000
4,0.122018,0.839174,250000


In [54]:
def square(x) -> ps.Series[np.float64]:
    return x ** 2

spdf.apply(square)

,A,B,C
0,0.387359,0.006970,10000.0
1,0.413511,0.378532,40000.0
2,0.375208,0.408194,90000.0
3,0.026713,0.989317,160000.0
4,0.122018,0.839174,250000.0


In [57]:
spdf_5 = ps.DataFrame({"A" : range(1000)})
spdf_5.apply(lambda col : col.max())

/usr/local/lib/python3.12/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


A    999
dtype: int64

### Groupping Data

In [58]:
psdf = ps.DataFrame({'A' : [1,2,2,3,4],
                     'B': [10,20,30,30,50],
                     'C' : [5,7,9,11,13]})

In [59]:
psdf.groupby("A").sum()

/usr/local/lib/python3.12/site-packages/pyspark/pandas/groupby.py:893: FutureWarning: Default value of `numeric_only` will be changed to `False` instead of `True` in 4.0.0.
  warnings.warn(


,B,C
A,,
1,10,5
2,50,16
3,30,11
4,50,13


In [60]:
psdf.groupby(["A","B"]).sum()

/usr/local/lib/python3.12/site-packages/pyspark/pandas/groupby.py:893: FutureWarning: Default value of `numeric_only` will be changed to `False` instead of `True` in 4.0.0.
  warnings.warn(


C
A B     
1 10   5
2 20   7
  30   9
3 30  11
4 50  13